# 3D feature extraction of Cell Painting performed on organoids 

This notebook aims to explain ans walkthrough what features are being extracted and how they are extracted.
These features are 3D features extracted from 3D image sets.
Where an image set is a collection of 3D images that are images of the same object but from using different light spectra.

An overview of how the features are extracted is shown in the following diagram:
![Feature Extraction Overview](../diagram/Featuization.png)

## The process
The process of featurization is adapted from a Cellprofiler pipeline approach and standard image-based profiling practices. 
The 3D image sets are segmented into objects:
* Organoid
* Nucleus
* Cell
* Cytoplasm

The objects are then used to extract features.
Where a feature is a measurement within the segmented object.
The feature extraction is performed using python libraries such as:
* [scikit-image](https://scikit-image.org/)
* [scipy](https://www.scipy.org/)
* [mahotas](https://mahotas.readthedocs.io/en/latest/)
* [numpy](https://numpy.org/)
* [cupy](https://docs.cupy.dev/en/stable/)
* [cucim](https://docs.rapids.ai/api/cucim/stable/)

The code is adapted and trimmed down to a more functional (as opposed to Object Oriented) approach from [Cellprofiler](https://cellprofiler.org/).
The code is adapted to reduce the number of list declarations and use generators instead.
The code is also written to be distributed across multiple CPU cores, and to be run on separate CPUs across image sets.
The idea is to run a single image set object - channel combination feature extraction method on a single CPU core.
In the case where a compute cluster is not available, the code has also been adapted to run on a single GPU for increased performance.

So what are the features we are extracting?

## Feature types:
While some feature types are quite intuitive, others are not.

In [1]:
import pathlib

import duckdb
import pandas as pd
from IPython.display import Markdown, display

In [2]:
path_to_db = pathlib.Path("../../data/NF0014/converted_profiles/C4-2.sqlite").resolve(
    strict=True
)
conn = duckdb.connect(path_to_db)

# read the schema
df = conn.execute("SELECT * FROM Nuclei").fetchdf()
columns = df.columns

In [3]:
# remove the object_id and image_set from the columns list
columns = [col for col in columns if col not in ["object_id", "image_set"]]
# make the columns a df
df = pd.DataFrame(columns)
df[
    [
        "feature_type",
        "Compartment",
        "Channel",
        "Meaurement",
        "Extra1",
        "Extra2",
        "Extra3",
    ]
] = df[0].str.split("_", expand=True)
df.drop(columns=["Extra1", "Extra2", "Extra3"], inplace=True)

In [4]:
# if the feature type is texture, then merge the compartment and channel columns into the measurement column
df.loc[df["feature_type"] == "Texture", "Meaurement"] = (
    df.loc[df["feature_type"] == "Texture", "Compartment"]
    + "_"
    + df.loc[df["feature_type"] == "Texture", "Channel"]
)
df.drop(columns=[0, "Compartment", "Channel"], inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
colocalization_feature_count = df.loc[df["feature_type"] == "Colocalization"].shape[0]
# the rest of the features count
rest_feature_count = df.shape[0] - colocalization_feature_count - 2  # 2 for neighbors
print(len(df))
areasizeshape_features = df.loc[df["feature_type"] == "Area.Size.Shape"].to_markdown(
    index=False
)
colocalization_features = df.loc[df["feature_type"] == "Colocalization"].to_markdown(
    index=False
)
intensity_features = df.loc[df["feature_type"] == "Intensity"].to_markdown(index=False)
granularity_features = df.loc[df["feature_type"] == "Granularity"].to_markdown(
    index=False
)
neighbors_features = df.loc[df["feature_type"] == "NEIGHBORS"].to_markdown(index=False)
texture_features = df.loc[df["feature_type"] == "Texture"].to_markdown(index=False)

95


### AreaSizeShape features

In [5]:
display(Markdown(areasizeshape_features))

| feature_type    | Meaurement          |
|:----------------|:--------------------|
| Area.Size.Shape | VOLUME              |
| Area.Size.Shape | CENTER.X            |
| Area.Size.Shape | CENTER.Y            |
| Area.Size.Shape | CENTER.Z            |
| Area.Size.Shape | BBOX.VOLUME         |
| Area.Size.Shape | MIN.X               |
| Area.Size.Shape | MAX.X               |
| Area.Size.Shape | MIN.Y               |
| Area.Size.Shape | MAX.Y               |
| Area.Size.Shape | MIN.Z               |
| Area.Size.Shape | MAX.Z               |
| Area.Size.Shape | EXTENT              |
| Area.Size.Shape | EULER.NUMBER        |
| Area.Size.Shape | EQUIVALENT.DIAMETER |
| Area.Size.Shape | SURFACE.AREA        |

The AreaSizeShape features are a collection of features that are related to the size and shape of the object.
These are calculated by generating a 3D mesh of the object and calculating the volume, surface area, and other shape features.
This is a mesh and voxel based approach.

### Colocalization features

In [6]:
display(Markdown(colocalization_features))

| feature_type   | Meaurement                     |
|:---------------|:-------------------------------|
| Colocalization | MEAN.CORRELATION.COEFF         |
| Colocalization | MEDIAN.CORRELATION.COEFF       |
| Colocalization | MIN.CORRELATION.COEFF          |
| Colocalization | MAX.CORRELATION.COEFF          |
| Colocalization | MEAN.MANDERS.COEFF.M1          |
| Colocalization | MEDIAN.MANDERS.COEFF.M1        |
| Colocalization | MIN.MANDERS.COEFF.M1           |
| Colocalization | MAX.MANDERS.COEFF.M1           |
| Colocalization | MEAN.MANDERS.COEFF.M2          |
| Colocalization | MEDIAN.MANDERS.COEFF.M2        |
| Colocalization | MIN.MANDERS.COEFF.M2           |
| Colocalization | MAX.MANDERS.COEFF.M2           |
| Colocalization | MEAN.OVERLAP.COEFF             |
| Colocalization | MEDIAN.OVERLAP.COEFF           |
| Colocalization | MIN.OVERLAP.COEFF              |
| Colocalization | MAX.OVERLAP.COEFF              |
| Colocalization | MEAN.K1                        |
| Colocalization | MEDIAN.K1                      |
| Colocalization | MIN.K1                         |
| Colocalization | MAX.K1                         |
| Colocalization | MEAN.K2                        |
| Colocalization | MEDIAN.K2                      |
| Colocalization | MIN.K2                         |
| Colocalization | MAX.K2                         |
| Colocalization | MEAN.MANDERS.COEFF.COSTES.M1   |
| Colocalization | MEDIAN.MANDERS.COEFF.COSTES.M1 |
| Colocalization | MIN.MANDERS.COEFF.COSTES.M1    |
| Colocalization | MAX.MANDERS.COEFF.COSTES.M1    |
| Colocalization | MEAN.MANDERS.COEFF.COSTES.M2   |
| Colocalization | MEDIAN.MANDERS.COEFF.COSTES.M2 |
| Colocalization | MIN.MANDERS.COEFF.COSTES.M2    |
| Colocalization | MAX.MANDERS.COEFF.COSTES.M2    |

For coloaclization features, we calulate the features by using two different sprectra and the same object.
Example:
* Organoid - DAPI and Phalloidin - calculate the colocalization features between the two spectra.

### Granularity features

In [7]:
display(Markdown(granularity_features))

| feature_type   | Meaurement    |
|:---------------|:--------------|
| Granularity    | GRANULARITY.1 |

Note that granularity is calulated over a granularity spectrum in this case from 1 to 16. 
Different features will be extracted at different granularity spectra.

### Intensity features

In [8]:
display(Markdown(intensity_features))

| feature_type   | Meaurement                |
|:---------------|:--------------------------|
| Intensity      | CM.X                      |
| Intensity      | CM.Y                      |
| Intensity      | CM.Z                      |
| Intensity      | CMI.X                     |
| Intensity      | CMI.Y                     |
| Intensity      | CMI.Z                     |
| Intensity      | DIFF.X                    |
| Intensity      | DIFF.Y                    |
| Intensity      | DIFF.Z                    |
| Intensity      | EDGE.COUNT                |
| Intensity      | I.X                       |
| Intensity      | I.Y                       |
| Intensity      | I.Z                       |
| Intensity      | INTEGRATED.INTENSITY      |
| Intensity      | INTEGRATED.INTENSITY.EDGE |
| Intensity      | LOWER.QUARTILE.INTENSITY  |
| Intensity      | MAD.INTENSITY             |
| Intensity      | MASS.DISPLACEMENT         |
| Intensity      | MAX.INTENSITY             |
| Intensity      | MAX.INTENSITY.EDGE        |
| Intensity      | MAX.X                     |
| Intensity      | MAX.Y                     |
| Intensity      | MAX.Z                     |
| Intensity      | MEAN.INTENSITY            |
| Intensity      | MEAN.INTENSITY.EDGE       |
| Intensity      | MEDIAN.INTENSITY          |
| Intensity      | MIN.INTENSITY             |
| Intensity      | MIN.INTENSITY.EDGE        |
| Intensity      | STD.INTENSITY             |
| Intensity      | STD.INTENSITY.EDGE        |
| Intensity      | UPPER.QUARTILE.INTENSITY  |
| Intensity      | VOLUME                    |

Note that the intensity features are extracted from the 3D voxel data.

### Neighbors features

In [9]:
display(Markdown(neighbors_features))

| feature_type   | Meaurement   |
|----------------|--------------|

### Texture features

In [10]:
display(Markdown(texture_features))

| feature_type   | Meaurement                                 |
|:---------------|:-------------------------------------------|
| Texture        | Angular.Second.Moment_256.1                |
| Texture        | Contrast_256.1                             |
| Texture        | Correlation_256.1                          |
| Texture        | Difference.Entropy_256.1                   |
| Texture        | Difference.Variance_256.1                  |
| Texture        | Entropy_256.1                              |
| Texture        | Information.Measure.of.Correlation.1_256.1 |
| Texture        | Information.Measure.of.Correlation.2_256.1 |
| Texture        | Inverse.Difference.Moment_256.1            |
| Texture        | Sum.Average_256.1                          |
| Texture        | Sum.Entropy_256.1                          |
| Texture        | Sum.Variance_256.1                         |
| Texture        | Variance_256.1                             |

Note for texture features there are two parameters for which texture features are calculated:
* The number of gray levels used to caculate the cooccurence matrix (here it is set to 256)
* The offset used to calculate the cooccurence matrix (here it is set to 1)

Using different offsets will yield different results and inform the user about different texture features.

There are $n_{features\_per\_object}$ different total features extracted for each object - channel combination.
Therefore we can say that the number of features extracted is:
### $$n_{features} = n_{objects} * n_{channels} * n_{features\_per\_object}$$
With the exception of the colocalization features which are extracted from a object - (channel - channel) combination.
### $$n_{features} = n_{objects} * \frac{n_{channels} * (n_{channels} - 1)}{2} * n_{features\_per\_object}$$
Where:
* #### $n_{features}$ is the total number of features extracted
* #### $n_{objects}$ is the number types of objects segmented e.g. Organoid, Nucleus, Cell, Cytoplasm
* #### $n_{channels}$ is the number of channels in the image 
* #### $n_{features\_per\_object}$ is the number of features extracted per object

In [11]:
n_channels = 5
n_object_types = 4
colocalization_features_total = (
    colocalization_feature_count
    * ((n_channels * (n_channels - 1)) / 2)
    * n_object_types
)
neighbors_features = 2
rest_feature_total = rest_feature_count * n_channels * n_object_types
total_features = int(
    colocalization_features_total + rest_feature_total + neighbors_features
)
print(f"Total theoretical features: {total_features}")

Total theoretical features: 2502
